## Multiple Chain 快速入门

Runnables 可以轻松地用来串联多个 Chains，使用 RunnablePassthrough 将输出同时传给多条后继链。

```
     Input
      / \
     /   \
 Chain1 Chain2
     \   /
      \ /
      Combine
```

本指南展示如何使用 Runnable 实现多个 AI 关于相同话题的辩论：

```
    输入话题
       |
       |
    原始观点
      / |\
     /  |  \
 正面论述| 反面论述
     \  | /
      \ |/
     最终总结
```

In [4]:
from dotenv import load_dotenv
import os
# 加载 .env 文件
load_dotenv()

# 从环境变量中读取 API 密钥和 API 基础 URL
openai_api_key = os.environ.get('OPENAI_API_KEY')
openai_api_base = os.environ.get('OPENAI_API_BASE')

In [2]:
# 导入相关模块，包括运算符、输出解析器、聊天模板、ChatOpenAI 和 运行器
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough



# 创建一个计划器，生成一个关于给定输入的论证
planner = (
    ChatPromptTemplate.from_template("生成关于以下内容的论点: {input}")
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

# 创建正面论证的处理链，列出关于基础回应的正面或有利的方面
arguments_for = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的正面或有利的方面"
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 创建反面论证的处理链，列出关于基础回应的反面或不利的方面
arguments_against = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的反面或不利的方面"
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 创建最终响应者，综合原始回应和正反论点生成最终的回应
final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "正面观点:\n{results_1}\n\n反面观点:\n{results_2}"),
            ("system", "给出批评后生成最终回应"),
        ]
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 构建完整的处理链，从生成论点到列出正反论点，再到生成最终回应
chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [3]:
print(chain.invoke({"input": "房地产低迷"}))

在探讨房地产市场低迷的现象时，既要考虑正面观点，也要关注反面观点，以便全面理解其影响和潜在机遇。以下是对上述正反观点的批评和最终回应：

### 对正面观点的批评

1. **购房者市场**：
   - 虽然低迷确实为购房者提供了机会，但并非所有购房者都能从中受益。经济不景气可能导致失业率上升，许多潜在购房者可能没有稳定的收入来源，进一步限制了他们的购房能力。

2. **市场调整与重组**：
   - 市场去泡沫的过程可能伴随大量投资者的损失和市场信心的下降，短期内可能导致更多的经济不稳定，而不是健康的市场发展。

3. **创新与转型**：
   - 虽然低迷可能催生创新，但并非所有创新都是积极的。某些新商业模式可能并不适合所有地区，甚至可能加剧市场的不平衡。

4. **改善基础设施**：
   - 政府推动基础设施建设虽然有潜力，但在低迷期，政府预算可能受到限制，导致基础设施投资不足，无法有效促进房地产市场复苏。

5. **租赁市场机会**：
   - 租金下降可能对租户有利，但对房东和投资者而言，可能会带来收入锐减，增加市场的不稳定性。

6. **投资机遇**：
   - 投资者在低迷期能够获得低价资产，但也面临更高的风险，未来回报的不确定性可能使得投资决策变得复杂。

### 对反面观点的批评

1. **经济因素的反面**：
   - 经济复苏并不是必然的，许多因素（如国际经济形势、政策干预等）都可能影响复苏的速度和强度。

2. **供需失衡的反面**：
   - 虽然需求可能在未来复苏，但供需失衡的根本问题若不解决，市场依然可能面临长期低迷。

3. **政策与监管的反面**：
   - 政策的有效性往往取决于实施情况，短期政策可能导致长期的市场扭曲，难以为市场健康发展提供保障。

4. **市场心理的反面**：
   - 市场信心的恢复是一个复杂过程，单靠政策或媒体的刺激不足以解决根本问题，消费者的信任需要时间建立。

5. **社会因素的反面**：
   - 虽然生活成本可能促使高端市场的发展，但对于大多数中低收入家庭而言，生活成本的上升依然是致命的负担，无法推动整体市场的健康发展。

6. **技术影响的反面**：
   - 线上交易和共享经济虽然带来便利，但也可能导致传统市场的萎缩，影响整体经济的稳定性。

### 最终回应

综合考虑

#### 流式输出

In [3]:
## chain 最终输出经过了 StrOutputParser 处理，所以可以直接输出流式输出 s
for s in chain.stream({"input": "全球经济"}):
    print(s, end="", flush=True)

在讨论全球经济的正面和反面观点时，我们可以看到各个方面的复杂性和相互关联性。以下是对以上论点的综合回应：

### 综合回应

1. **全球化的复杂性**：全球化确实带来了经济增长和文化交流，但也伴随着文化同质化和环境问题。为了实现经济与文化的双赢，各国需要在全球化的过程中，重视保护和弘扬地方文化，同时制定更严格的环境保护政策，以确保可持续发展。

2. **科技进步与包容性**：科技的进步是推动生产力提升的重要因素，但数字鸿沟和就业不稳定是不可忽视的挑战。各国应加强教育和技能培训，确保所有人都能平等地享受到科技进步的红利，从而减少社会不平等和不安定性。

3. **可持续发展的双重目标**：在追求经济增长的同时，必须重视环境保护。各国应采取更有效的环境政策，平衡经济发展与生态保护，避免资源争夺引发的地缘政治紧张。

4. **货币政策的审慎管理**：货币政策在促进经济增长方面发挥了重要作用，但也带来了金融不稳定和不平等加剧的问题。各国央行需要在刺激经济和控制风险之间找到平衡，确保货币政策的可持续性。

5. **新兴市场的机遇与挑战**：新兴市场的崛起为全球经济注入活力，但也带来了对传统经济体的依赖风险。发达国家应与新兴市场建立更紧密的合作关系，以应对全球经济波动带来的挑战。

6. **贸易政策的协调**：保护主义政策虽然短期内可能保护某些行业，但长期来看会损害全球经济的整体发展。各国应积极推动自由贸易，促进国际合作，以实现共同繁荣。

7. **疫情后的复苏策略**：疫情后的经济复苏确实存在不平衡的问题，各国应采取灵活的财政政策，优先支持受影响严重的行业和地区，确保经济复苏的包容性和可持续性。

### 结论

全球经济是一个复杂的系统，各国在追求自身利益的同时，必须加强合作与协调，以应对共同面临的挑战。通过创新、教育和可持续发展，世界各国可以在经济增长的基础上，推动更公平和包容的全球经济体系。关键在于各国如何在面对机遇与挑战时，采取适当的政策和措施，以实现长远的可持续发展。

### Homework: 实现一个多链版本的代码生成，输入功能需求，输出2种（Python，Java）以上编程语言的代码实现。